#导入电影数据

In [152]:
#-*- coding:utf-8 -*-
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
movies_df=pd.read_csv('movies.csv')
movies_df.shape

(10866, 21)

In [153]:
movies_df=movies_df.drop_duplicates()
movies_df.shape

(10865, 21)

In [154]:
movies_df.head()

,id,imdb_id,popularity,budget,revenue,original_title,cast,homepage,director,tagline,...,overview,runtime,genres,production_companies,release_date,vote_count,vote_average,release_year,budget_adj,revenue_adj
0,135397,tt0369610,32.985763,150000000,1513528810,Jurassic World,Chris Pratt|Bryce Dallas Howard|Irrfan Khan|Vi...,http://www.jurassicworld.com/,Colin Trevorrow,The park is open.,...,Twenty-two years after the events of Jurassic ...,124,Action|Adventure|Science Fiction|Thriller,Universal Studios|Amblin Entertainment|Legenda...,2015-06-09,5562,6.5,2015,1.379999e+08,1.392446e+09
1,76341,tt1392190,28.419936,150000000,378436354,Mad Max: Fury Road,Tom Hardy|Charlize Theron|Hugh Keays-Byrne|Nic...,http://www.madmaxmovie.com/,George Miller,What a Lovely Day.,...,An apocalyptic story set in the furthest reach...,120,Action|Adventure|Science Fiction|Thriller,Village Roadshow Pictures|Kennedy Miller Produ...,2015-05-13,6185,7.1,2015,1.379999e+08,3.481613e+08
2,262500,tt2908446,13.112507,110000000,295238201,Insurgent,Shailene Woodley|Theo James|Kate Winslet|Ansel...,http://www.thedivergentseries.movie/#insurgent,Robert Schwentke,One Choice Can Destroy You,...,Beatrice Prior must confront her inner demons ...,119,Adventure|Science Fiction|Thriller,Summit Entertainment|Mandeville Films|Red Wago...,2015-03-18,2480,6.3,2015,1.012000e+08,2.716190e+08
3,140607,tt2488496,11.173104,200000000,2068178225,Star Wars: The Force Awakens,Harrison Ford|Mark Hamill|Carrie Fisher|Adam D...,http://www.starwars.com/films/star-wars-episod...,J.J. Abrams,Every generation has a story.,...,Thirty years after defeating the Galactic Empi...,136,Action|Adventure|Science Fiction|Fantasy,Lucasfilm|Truenorth Productions|Bad Robot,2015-12-15,5292,7.5,2015,1.839999e+08,1.902723e+09
4,168259,tt2820852,9.335014,190000000,1506249360,Furious 7,Vin Diesel|Paul Walker|Jason Statham|Michelle ...,http://www.furious7.com/,James Wan,Vengeance Hits Home,...,Deckard Shaw seeks revenge against Dominic Tor...,137,Action|Crime|Thriller,Universal Pictures|Original Film|Media Rights ...,2015-04-01,2947,7.3,2015,1.747999e+08,1.385749e+09


#根据问题筛选要研究的数据列  
问题1：电影类型是如何随着时代变化而变化的?  
问题2：环球影业和派拉蒙影业的电影之前数据指标有什么区别?  
问题3：和非小说改编的电影相比，基于小说改编的电影表现得怎么样?  
问题4：投资与收益以及电影评分间有什么联系?
对问题1，需要提取id,genres,release_year  

In [155]:
genres_df=movies_df['genres'].str.split('|',expand=True)
genres_df['id']=movies_df['id']
genres_df.head()

,0,1,2,3,4,id
0,Action,Adventure,Science Fiction,Thriller,None,135397
1,Action,Adventure,Science Fiction,Thriller,None,76341
2,Adventure,Science Fiction,Thriller,None,None,262500
3,Action,Adventure,Science Fiction,Fantasy,None,140607
4,Action,Crime,Thriller,None,None,168259


In [156]:
merged_df=movies_df.merge(genres_df,on='id')
merged_df[0].head()

0       Action
1       Action
2    Adventure
3       Action
4       Action
Name: 0, dtype: object

In [157]:
melted_df=pd.melt(merged_df,id_vars=['id','release_year'],value_vars=[0,1,2,3,4],value_name='movie_type')
melted_df=melted_df.drop('variable',axis=1).dropna(axis=0,how='any')
melted_df.head()

,id,release_year,movie_type
0,135397,2015,Action
1,76341,2015,Action
2,262500,2015,Adventure
3,140607,2015,Action
4,168259,2015,Action


In [158]:
melted_df.to_csv('id_year_genre.csv',index=False)

对于问题2，需要id,production_companies

In [159]:
companies_df=movies_df['production_companies'].str.split('|',expand=True)
companies_df.head()

,0,1,2,3,4
0,Universal Studios,Amblin Entertainment,Legendary Pictures,Fuji Television Network,Dentsu
1,Village Roadshow Pictures,Kennedy Miller Productions,None,None,None
2,Summit Entertainment,Mandeville Films,Red Wagon Entertainment,NeoReel,None
3,Lucasfilm,Truenorth Productions,Bad Robot,None,None
4,Universal Pictures,Original Film,Media Rights Capital,Dentsu,One Race Films


In [160]:
companies_df['id']=movies_df['id']
merged_df=movies_df.merge(companies_df,on='id')
melted_df=pd.melt(merged_df,id_vars=['id'],value_vars=[0,1,2,3,4],value_name='company_name')
melted_df=melted_df.drop('variable',axis=1).dropna(axis=0,how='any')
melted_df.head()

,id,company_name
0,135397,Universal Studios
1,76341,Village Roadshow Pictures
2,262500,Summit Entertainment
3,140607,Lucasfilm
4,168259,Universal Pictures


In [161]:
melted_df.to_csv('id_company.csv',index=False)

对问题3，需要筛选出keywords中含有based on novel的电影

In [162]:
novels_df=movies_df['keywords'].str.contains('based on novel')
movies_df['novel']=novels_df
novels_df=movies_df.loc[:,['id','popularity','revenue','budget','vote_count','vote_average','budget_adj','revenue_adj','novel']].dropna(axis=0,how='any')
novels_df.to_csv('novels.csv',index=False)

对问题4，筛选电影投资，收益，评分，流行度

In [163]:
movies_df.head()

,id,imdb_id,popularity,budget,revenue,original_title,cast,homepage,director,tagline,...,runtime,genres,production_companies,release_date,vote_count,vote_average,release_year,budget_adj,revenue_adj,novel
0,135397,tt0369610,32.985763,150000000,1513528810,Jurassic World,Chris Pratt|Bryce Dallas Howard|Irrfan Khan|Vi...,http://www.jurassicworld.com/,Colin Trevorrow,The park is open.,...,124,Action|Adventure|Science Fiction|Thriller,Universal Studios|Amblin Entertainment|Legenda...,2015-06-09,5562,6.5,2015,1.379999e+08,1.392446e+09,False
1,76341,tt1392190,28.419936,150000000,378436354,Mad Max: Fury Road,Tom Hardy|Charlize Theron|Hugh Keays-Byrne|Nic...,http://www.madmaxmovie.com/,George Miller,What a Lovely Day.,...,120,Action|Adventure|Science Fiction|Thriller,Village Roadshow Pictures|Kennedy Miller Produ...,2015-05-13,6185,7.1,2015,1.379999e+08,3.481613e+08,False
2,262500,tt2908446,13.112507,110000000,295238201,Insurgent,Shailene Woodley|Theo James|Kate Winslet|Ansel...,http://www.thedivergentseries.movie/#insurgent,Robert Schwentke,One Choice Can Destroy You,...,119,Adventure|Science Fiction|Thriller,Summit Entertainment|Mandeville Films|Red Wago...,2015-03-18,2480,6.3,2015,1.012000e+08,2.716190e+08,True
3,140607,tt2488496,11.173104,200000000,2068178225,Star Wars: The Force Awakens,Harrison Ford|Mark Hamill|Carrie Fisher|Adam D...,http://www.starwars.com/films/star-wars-episod...,J.J. Abrams,Every generation has a story.,...,136,Action|Adventure|Science Fiction|Fantasy,Lucasfilm|Truenorth Productions|Bad Robot,2015-12-15,5292,7.5,2015,1.839999e+08,1.902723e+09,False
4,168259,tt2820852,9.335014,190000000,1506249360,Furious 7,Vin Diesel|Paul Walker|Jason Statham|Michelle ...,http://www.furious7.com/,James Wan,Vengeance Hits Home,...,137,Action|Crime|Thriller,Universal Pictures|Original Film|Media Rights ...,2015-04-01,2947,7.3,2015,1.747999e+08,1.385749e+09,False


In [164]:
director_df=movies_df['director'].str.split('|',expand=True)
director_df['id']=movies_df['id']
merged_df=movies_df.merge(director_df,on='id')
melted_df=merged_df.melt(id_vars=['id','popularity','revenue','budget','vote_count','vote_average','budget_adj','revenue_adj','cast'],value_vars=[0,1,2,3,4],value_name='directors')
melted_df=melted_df.drop('variable',axis=1).dropna(axis=0,how='any')

cast_df=melted_df['cast'].str.split('|',expand=True)
cast_df[['id','directors']]=melted_df[['id','directors']]
merged_df2=melted_df.merge(cast_df,on=['id','directors'])
melted_df2=merged_df2.melt(id_vars=['id','popularity','revenue','budget','vote_count','vote_average','budget_adj','revenue_adj','directors'],value_vars=[0,1,2,3,4],value_name='casts')
melted_df2=melted_df2.drop('variable',axis=1).dropna(axis=0,how='any')
melted_df2.head()

,id,popularity,revenue,budget,vote_count,vote_average,budget_adj,revenue_adj,directors,casts
0,135397,32.985763,1513528810,150000000,5562,6.5,1.379999e+08,1.392446e+09,Colin Trevorrow,Chris Pratt
1,76341,28.419936,378436354,150000000,6185,7.1,1.379999e+08,3.481613e+08,George Miller,Tom Hardy
2,262500,13.112507,295238201,110000000,2480,6.3,1.012000e+08,2.716190e+08,Robert Schwentke,Shailene Woodley
3,140607,11.173104,2068178225,200000000,5292,7.5,1.839999e+08,1.902723e+09,J.J. Abrams,Harrison Ford
4,168259,9.335014,1506249360,190000000,2947,7.3,1.747999e+08,1.385749e+09,James Wan,Vin Diesel


In [165]:
#merged_df[merged_df['id']==9355]
#director_df[director_df['id']==42194]

In [169]:
melted_df.to_csv('directors.csv',index=False)
cast_df=movies_df['cast'].str.split('|',expand=True)
cast_df['id']=movies_df['id']
merged_df1=movies_df.merge(cast_df,on='id')
melted_df1=merged_df1.melt(id_vars=['id','popularity','revenue','budget','vote_count','vote_average','budget_adj','revenue_adj'],value_vars=[0,1,2,3,4],value_name='casts')
melted_df1=melted_df1.drop('variable',axis=1).dropna(axis=0,how='any')
melted_df1.to_csv('casts.csv')

In [167]:
melted_df2.to_csv('director_cast.csv',index=False)

参考资料:
多值字段处理 https://discussions.youdaxue.com/t/tableau/42153/2  
http://pandas.pydata.org/pandas-docs/stable/api.html#string-handling  
http://pandas.pydata.org/pandas-docs/stable/api.html#id9  
https://pandas.pydata.org/pandas-docs/stable/generated/pandas.melt.html  